# Cognitive Action Probe Testing

Test your trained cognitive action probes on text.

**Key Feature**: Automatically loads the best-performing layer for each action.

In [1]:
# IMPORTANT: Set AMD GPU environment variables BEFORE importing torch
import os
import subprocess

def detect_amd_gpu():
    try:
        result = subprocess.run(['lspci'], capture_output=True, text=True, timeout=2)
        return 'Advanced Micro Devices' in result.stdout
    except:
        return False

if detect_amd_gpu():
    print("AMD GPU detected - configuring ROCm environment variables")
    # PyTorch is compiled for gfx1100, not gfx1101
    # Override to use gfx1100 kernels for gfx1101 GPU (RX 7700/7800 XT)
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"
    os.environ["HIP_VISIBLE_DEVICES"] = "0"
    os.environ["AMD_SERIALIZE_KERNEL"] = "3"
    os.environ["TORCH_USE_HIP_DSA"] = "1"
    os.environ["PYTORCH_ROCM_ARCH"] = "gfx1100"  # Use gfx1100 kernels (closest match)
    os.environ["PYTORCH_HIP_ALLOC_CONF"] = "expandable_segments:True"
    os.environ["HIP_LAUNCH_BLOCKING"] = "1"  # Synchronous execution for better errors
    print(f"  HSA_OVERRIDE_GFX_VERSION: {os.environ['HSA_OVERRIDE_GFX_VERSION']}")
    print(f"  PYTORCH_ROCM_ARCH: {os.environ['PYTORCH_ROCM_ARCH']}")
    print("  Note: Using gfx1100 kernels for gfx1101 GPU")
else:
    print("No AMD GPU detected")

AMD GPU detected - configuring ROCm environment variables
  HSA_OVERRIDE_GFX_VERSION: 11.0.0
  PYTORCH_ROCM_ARCH: gfx1100
  Note: Using gfx1100 kernels for gfx1101 GPU


In [2]:
import sys
from pathlib import Path
import torch

# Add src/probes to path
sys.path.insert(0, str(Path.cwd() / 'src' / 'probes'))

from best_multi_probe_inference import BestMultiProbeInferenceEngine
from best_probe_loader import print_performance_summary

AMD GPU detected - configuring ROCm environment variables
  HSA_OVERRIDE_GFX_VERSION: 11.0.0
  PYTORCH_ROCM_ARCH: gfx1100
  TORCH_USE_HIP_DSA: 1
  HIP_LAUNCH_BLOCKING: 1


/home/koalacrown/Desktop/Code/Projects/brije/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. View Probe Performance

In [3]:
PROBES_BASE_DIR = Path('data/probes_binary')
print_performance_summary(PROBES_BASE_DIR, top_n=45)

PROBE PERFORMANCE SUMMARY

Top 45 Best Performing Probes (by AUC):
----------------------------------------------------------------------
 1. suspending_judgment                 Layer 21  AUC: 0.9995  F1: 0.9347
 2. accepting                           Layer 22  AUC: 0.9971  F1: 0.8601
 3. zooming_in                          Layer 23  AUC: 0.9961  F1: 0.8421
 4. analogical_thinking                 Layer 22  AUC: 0.9957  F1: 0.8466
 5. counterfactual_reasoning            Layer 22  AUC: 0.9937  F1: 0.8912
 6. zooming_out                         Layer 23  AUC: 0.9934  F1: 0.9200
 7. divergent_thinking                  Layer 21  AUC: 0.9897  F1: 0.8168
 8. attentional_deployment              Layer 21  AUC: 0.9892  F1: 0.7475
 9. concretizing                        Layer 22  AUC: 0.9882  F1: 0.7957
10. perspective_taking                  Layer 21  AUC: 0.9846  F1: 0.6480
11. distinguishing                      Layer 21  AUC: 0.9821  F1: 0.6947
12. hypothesis_generation               Layer 25

## 2. Load Inference Engine

In [4]:
MODEL_NAME = 'google/gemma-3-4b-it'

print('Loading probes...')
engine = BestMultiProbeInferenceEngine(
    probes_base_dir=PROBES_BASE_DIR,
    model_name=MODEL_NAME
)
print('Ready!')

Loading probes...
Initializing BestMultiProbeInferenceEngine...
  Probes base dir: data/probes_binary
  Model: google/gemma-3-4b-it
  Device: cuda

Loading best probes for 45 actions...
Device: cuda

Loaded probe from data/probes_binary/layer_21/probe_abstracting.pth
Loaded probe from data/probes_binary/layer_22/probe_accepting.pth
Loaded probe from data/probes_binary/layer_22/probe_analogical_thinking.pth
Loaded probe from data/probes_binary/layer_23/probe_analyzing.pth
Loaded probe from data/probes_binary/layer_22/probe_applying.pth
Loaded probe from data/probes_binary/layer_21/probe_attentional_deployment.pth
Loaded probe from data/probes_binary/layer_22/probe_cognition_awareness.pth
Loaded probe from data/probes_binary/layer_22/probe_concretizing.pth
Loaded probe from data/probes_binary/layer_21/probe_connecting.pth
Loaded probe from data/probes_binary/layer_22/probe_convergent_thinking.pth
  Loaded 10 probes...
Loaded probe from data/probes_binary/layer_22/probe_counterfactual_rea

/home/koalacrown/Desktop/Code/Projects/brije/src/probes/probe_models.py:419: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(load_path, map_location=device)

Detected vision-language model. Loading text-only (skipping vision tower)...


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.38s/it]



✓ Initialized with 45 probes across 6 layers

Ready!


## 3. Test on Your Text

In [6]:
text = '''After receiving feedback, I began reconsidering my approach.
I realized I had been making assumptions without fully understanding the constraints.'''

predictions = engine.predict(text, top_k=10, threshold=0.1)

print('Detected Cognitive Actions:')
print('='*70)
for i, pred in enumerate(predictions, 1):
    marker = '✓' if pred.is_active else '○'
    print(f"\nSentence: {text}")
    print(f"{marker} {i:2d}. {pred.action_name:30s} {pred.confidence:6.1%}")
    print(f"      (Layer {pred.layer}, AUC: {pred.auc:.3f})")
    if hasattr(pred, 'beliefs') and pred.beliefs:
        print("      Beliefs:")
        for belief in pred.beliefs:
            print(f"        - {belief}")

Detected Cognitive Actions:

Sentence: After receiving feedback, I began reconsidering my approach.
I realized I had been making assumptions without fully understanding the constraints.
✓  1. updating_beliefs               100.0%
      (Layer 25, AUC: 0.935)

Sentence: After receiving feedback, I began reconsidering my approach.
I realized I had been making assumptions without fully understanding the constraints.
✓  2. metacognitive_regulation        35.5%
      (Layer 22, AUC: 0.929)


## 4. Compare Two Texts

In [7]:
text1 = 'Analyzing the data to identify patterns and trends.'
text2 = 'Brainstorming creative solutions to the problem.'

comparison = engine.compare_texts(text1, text2, top_k=5)

print('TEXT 1:', text1)
print('\nTop actions:')
for action, conf in comparison['text1_top_actions'][:5]:
    print(f'  - {action:30s} {conf:.1%}')

print('\n' + '='*70)
print('TEXT 2:', text2)
print('\nTop actions:')
for action, conf in comparison['text2_top_actions'][:5]:
    print(f'  - {action:30s} {conf:.1%}')

TEXT 1: Analyzing the data to identify patterns and trends.

Top actions:
  - emotion_receiving              0.0%
  - emotion_responding             0.0%
  - abstracting                    0.0%
  - applying                       0.0%
  - evaluating                     0.0%

TEXT 2: Brainstorming creative solutions to the problem.

Top actions:
  - divergent_thinking             99.6%
  - emotion_responding             0.0%
  - applying                       0.0%
  - creating                       0.0%
  - metacognitive_regulation       0.0%


## 5. Batch Processing

In [14]:
texts = [
    "The quarterly numbers look... interesting. Revenue up 12%, but margins down 3%. Customer acquisition costs rising while retention rates plateau. Something doesn't add up here.",
    "What if we completely flipped the script? Instead of chasing the same customers everyone else wants, what about targeting the segment nobody's paying attention to?",
    "Last quarter's campaign... we spent $50K on social media ads, got 200 signups, but only 15 converted. That's a 7.5% conversion rate. Industry average is 12%. We're bleeding money.",
    "That client meeting keeps replaying in my head. Sarah said 'the integration feels clunky' and I brushed it off. Now three clients have mentioned the same thing. I should have listened.",
    "My brain is scattered. Need to organize this mess: finish the Q4 budget review, prep for tomorrow's board meeting, and draft the hiring plan for next quarter. Otherwise I'll forget something crucial.",
    "Where are we on the Johnson account? Last I heard, legal was reviewing the contract. Marketing said they'd have the campaign ready by Friday. Finance needs the numbers by end of week. Everything's converging.",
    "Client feedback from Project Alpha, user research from Beta, and market analysis from Gamma. All pointing in different directions. There's a pattern here I'm not seeing yet.",
    "Option A: expand to Europe, higher risk but potentially 40% revenue growth. Option B: focus on domestic market, safer but maybe 15% growth. Both have merit. Both have downsides.",
    "The website keeps crashing during peak hours. Server logs show increased traffic, but that shouldn't cause failures. There's something else going on.",
    "The interns are lost. I'm throwing terms like 'conversion funnel' and 'attribution modeling' at them. They need the basics first - what we're trying to achieve and why.",
    "This product launch strategy feels incomplete. Maybe I should bounce ideas off the team. Fresh perspectives could reveal blind spots I'm missing.",
    "I've been assuming our target demographic is 25-35 year olds. But what if that's wrong? What if I'm basing decisions on outdated assumptions?",
    "This dashboard is overwhelming. Revenue charts, user engagement metrics, conversion rates, churn analysis. Too much noise. Need to focus on what actually matters.",
    "The current approach isn't working. Users aren't engaging with the new feature. Maybe we need to pivot. Try a different angle entirely.",
    "These customer segments look similar on paper - both tech-savvy, both high income. But their behavior patterns are completely different. What am I missing?",
    "If we launch in Q2 instead of Q1, we'd have more time for testing. But competitors might beat us to market. If we rush Q1, we risk bugs. If we wait, we risk irrelevance.",
    "The manager's email was vague: 'streamline the process.' What does that mean exactly? Reduce steps? Automate tasks? Cut costs? Need to clarify before I act.",
    "I'm recommending we increase the marketing budget by 30%. But why? Because last quarter's campaign worked? Because competitors are spending more? Need solid reasoning.",
    "Sally flagged that our pricing model doesn't account for seasonal fluctuations. She's right. Our revenue projections assume steady demand year-round. That's unrealistic.",
    "The project timeline is chaotic. Phase 1 should inform Phase 2, which should inform Phase 3. But everything's happening simultaneously. Need to map out dependencies.",
    "This market research feels biased. The methodology seems sound, but the conclusions feel predetermined. Like they found what they were looking for.",
    "The correlation between social media engagement and sales is strong. But that doesn't mean social media causes sales. Could be reverse causation, or a third factor entirely.",
    "Let's test this hypothesis: if our target users really want this feature, they'll use it within the first week. If not, we'll know it's not solving a real problem.",
    "Both theories explain the data well. Theory A focuses on user behavior, Theory B on market conditions. They're not mutually exclusive, but they emphasize different factors.",
    "This industry report cites impressive statistics, but I don't recognize the research firm. Need to verify their credibility before I base any decisions on their findings.",
    "Today's priorities are overwhelming. The client presentation, the budget review, the team meeting, the product demo. Can't do everything. Need to pick what's truly urgent.",
    "The alternative approach might be better. Current method is familiar, but the new one could be more efficient. Should we compare them side by side before deciding?",
    "Let me explain this simply: we're not making money because we're spending more to acquire customers than we earn from them. Like buying a $10 item for $15.",
    "I remember being overwhelmed by all these metrics and KPIs when I started. Jamie looks lost in the same way. Maybe I can help them understand what actually matters.",
    "Sitting here watching people interact with our app. Some scroll quickly, others pause and tap. Some get frustrated and leave. Others seem to find what they need. Patterns emerging."
]

batch_results = engine.predict_batch(texts, top_k=10, threshold=0.0001)

for i, (text, preds) in enumerate(zip(texts, batch_results), 1):
    print(f'\n{i}. {text}')
    for j, pred in enumerate(preds, 1):
        print(f'   {j}. {pred.action_name:30s} {pred.confidence*100:.8f}% (L{pred.layer})')


1. The quarterly numbers look... interesting. Revenue up 12%, but margins down 3%. Customer acquisition costs rising while retention rates plateau. Something doesn't add up here.
   1. noticing                       100.00000000% (L23)
   2. zooming_in                     0.31738281% (L23)

2. What if we completely flipped the script? Instead of chasing the same customers everyone else wants, what about targeting the segment nobody's paying attention to?
   1. divergent_thinking             100.00000000% (L21)
   2. creating                       0.17013550% (L21)

3. Last quarter's campaign... we spent $50K on social media ads, got 200 signups, but only 15 converted. That's a 7.5% conversion rate. Industry average is 12%. We're bleeding money.
   1. noticing                       100.00000000% (L23)
   2. evaluating                     8.64257812% (L23)

4. That client meeting keeps replaying in my head. Sarah said 'the integration feels clunky' and I brushed it off. Now three client